In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la

import pyopencl as cl
from pytools import add_tuples

import sumpy.toys as t
from sumpy.expansion.local import VolumeTaylorLocalExpansion
from sumpy.expansion.multipole import VolumeTaylorMultipoleExpansion
from sumpy.kernel import HelmholtzKernel, LaplaceKernel, YukawaKernel  # noqa: F401


rng = np.random.default_rng(seed=42)
order = 4

if 0:
    knl = LaplaceKernel(2)
    pde = [(1, (2, 0)), (1, (0, 2))]
    extra_kernel_kwargs = {}

else:
    helm_k = 1.2
    knl = HelmholtzKernel(2)
    extra_kernel_kwargs = {"k": helm_k}

    pde = [(1, (2, 0)), (1, (0, 2)), (helm_k**2, (0, 0))]

mpole_expn = VolumeTaylorMultipoleExpansion(knl, order)
local_expn = VolumeTaylorLocalExpansion(knl, order)

cl_ctx = cl.create_some_context(answers=["port"])

tctx = t.ToyContext(
    cl_ctx,
    knl,
    mpole_expn_class=type(mpole_expn),
    local_expn_class=type(local_expn),
    extra_kernel_kwargs=extra_kernel_kwargs,
)

In [ ]:
pt_src = t.PointSources(tctx, rng.uniform(-0.5, 0.5, size=(2, 50)), np.ones(50))

mexp = t.multipole_expand(pt_src, [0, 0], order)

In [ ]:
mexp.coeffs

In [ ]:
def build_pde_mat(expn, pde):
    coeff_ids = expn.get_coefficient_identifiers()
    id_to_index = expn._storage_loc_dict

    # FIXME: specific to scalar PDEs
    pde_mat = np.zeros((len(coeff_ids), len(coeff_ids)))

    row = 0
    for base_coeff_id in coeff_ids:
        valid = True

        for pde_coeff, coeff_id_offset in pde:
            other_coeff = add_tuples(base_coeff_id, coeff_id_offset)
            if other_coeff not in id_to_index:
                valid = False
                break

            pde_mat[row, id_to_index[other_coeff]] = pde_coeff

        if valid:
            row += 1
        else:
            pde_mat[row] = 0

    return pde_mat[:row]


pde_mat = build_pde_mat(mpole_expn, pde)

In [ ]:
def find_nullspace(mat, tol=1e-10):
    _u, sig, vt = la.svd(pde_mat, full_matrices=True)
    zerosig = np.where(np.abs(sig) < tol)[0]
    if zerosig.size:
        nullsp_start = zerosig[0]
        assert np.array_equal(zerosig, np.arange(nullsp_start, pde_mat.shape[1]))
    else:
        nullsp_start = pde_mat.shape[0]

    return vt[nullsp_start:].T


nullsp = find_nullspace(pde_mat)

In [ ]:
la.norm(pde_mat @ nullsp)

In [ ]:
def build_translation_mat(mexp, to_center):
    n = len(mexp.coeffs)
    result = np.zeros((n, n))

    for j in range(n):
        unit_coeffs = np.zeros(n)
        unit_coeffs[j] = 1
        unit_mexp = mexp.with_coeffs(unit_coeffs)

        result[:, j] = t.multipole_expand(unit_mexp, to_center).coeffs

    return result


new_center = np.array([0, 0.5])
tmat = build_translation_mat(mexp, new_center)

In [ ]:
plt.imshow(tmat)

In [ ]:
nullsp.shape

In [ ]:
if 0:
    reduction_mat = nullsp.T
    expansion_mat = nullsp
elif 1:
    chosen_indices_and_coeff_ids = [
        (i, cid)
        for i, cid in enumerate(mpole_expn.get_coefficient_identifiers())
        if cid[0] < 2
    ]
    chosen_indices = [idx for idx, _ in chosen_indices_and_coeff_ids]

    expansion_mat = np.zeros((
        len(mpole_expn.get_coefficient_identifiers()),
        len(chosen_indices_and_coeff_ids),
    ))
    for i, (idx, _) in enumerate(chosen_indices_and_coeff_ids):
        expansion_mat[idx, i] = 1

    reduction_mat = (nullsp @ la.inv(nullsp[chosen_indices])).T

In [ ]:
def plot_coeffs(expn, coeffs, **kwargs):
    x = [cid[0] for cid in expn.get_coefficient_identifiers()]
    y = [cid[1] for cid in expn.get_coefficient_identifiers()]
    plt.scatter(x, y, c=coeffs, **kwargs)
    plt.colorbar()

    for cid, coeff in zip(expn.get_coefficient_identifiers(), coeffs, strict=True):
        plt.text(cid[0], cid[1] + 0.2, f"{coeff:.1f}")

In [ ]:
proj_mexp = mexp.with_coeffs(expansion_mat @ reduction_mat @ mexp.coeffs)

proj_resid = proj_mexp.coeffs - mexp.coeffs

plot_coeffs(mpole_expn, np.log10(1e-15 + np.abs(proj_resid)), vmin=-15, vmax=2)

In [ ]:
print(t.l_inf(proj_mexp - mexp, 1.2, center=[3, 0]))

In [ ]:
trans_unproj = t.multipole_expand(mexp, new_center)
trans_proj = t.multipole_expand(proj_mexp, new_center)

print(t.l_inf(trans_unproj - trans_proj, 1.2, center=[3, 0]))

In [ ]:
print(trans_proj.coeffs - trans_unproj.coeffs)

In [ ]:
la.norm(reduction_mat @ (trans_proj.coeffs - trans_unproj.coeffs))

In [ ]:
t.l_inf(trans_unproj - pt_src, 1.2, center=[3, 0])

In [ ]:
t.l_inf(mexp - pt_src, 1.2, center=[3, 0])